In [1]:
# Structurally Contrained Recurrent Network (SCRN) Model
#
# This gives an implementation of the SCRN model given in Mikolov et al. 2015, arXiv:1412.7753 [cs.NE], 
# https://arxiv.org/abs/1412.7753 using Python and Tensorflow.
#
# This IPython Notebook provides an example of how to call the associated library of Python scripts.  
# Mikolov et al. should be consulted to make sure of the correct hyperparameter values.
#
# Stuart Hagler, 2017

In [2]:
# Imports
import math
import sys

# Local Imports
sys.path.insert(0, 'python')
from lstm import lstm_graph
from read_data import read_data
from scrn import scrn_graph
from srn import srn_graph
from tokens import text_elements_to_tokens

In [3]:
# Flags
rnn_flg = 3      # 1 for SRN
                 # 2 for LSTM
                 # 3 for SCRN
usecase_flg = 2  # 1 for predicting letters
                 # 2 for predicting words with cutoff for infrequent words

In [4]:
# Network-specific hyperparameters
if rnn_flg == 1:
    
    # Network hyperparameters
    hidden_size = 110               # Dimension of the hidden vector
    
    # Training hyperparameters
    num_training_unfoldings = 10    # Number of training unfoldings
    
elif rnn_flg == 2:
    
    # Network hyperparameters
    hidden_size = 110               # Dimension of the hidden vector
    
    # Training hyperparameters
    num_training_unfoldings = 10    # Number of training unfoldings
    
elif rnn_flg == 3:
    
    # Network hyperparameters
    alpha = 0.95                    #
    hidden_size = 100               # Dimension of the hidden vector
    state_size = 10                 # Dimension of the state vector

    # Training hyperparameters
    num_training_unfoldings = 50    # Number of training unfoldings
    
# General network hyperparameters
word_frequency_cutoff = 10          # Cutoff for infrequent words for usecase_flg = 2

# General training hyperparameters
base_batch_size = 32                # Batch size across all towers
clip_norm = 1.25                    # Norm for gradient clipping
learning_decay = 1/1.5              # Multiplier to decay the learn rate when required
learning_rate = 0.05                # Initial learning rate
momentum = 0.9                      # Momentum for training
num_epochs = 50                     # Total number of epochs to run the algorithm
num_validation_unfoldings = 1000    # Number of validation unfoldings
optimization_frequency = 5          # Number of unfoldings before optimization step
summary_frequency = 500             # Summary information is displayed after training this many batches

# Cluster
cluster_spec = { "worker" : [ "10.171.141.101:2222",
                              "10.171.141.118:2223", 
                              "10.171.141.122:2224" ] }
num_gpus = [ 2, 2, 2 ]              # Number of GPUs available for each worker host (must agree with cluster_spec)

# Data file
filename = '/opt/pynb/text8.zip'

In [ ]:
# Prepare training, validation, test data sets
num_towers = sum(num_gpus)
batch_size = base_batch_size // num_towers
raw_data = read_data(usecase_flg, filename)
data, dictionary, reverse_dictionary, vocabulary_size = text_elements_to_tokens(usecase_flg, raw_data, 
                                                                                word_frequency_cutoff)
training_size = math.floor((9/11)*len(raw_data)/num_towers)
validation_size = math.floor((1/11)*len(raw_data)/num_towers)
test_size = math.floor((1/11)*len(raw_data)/num_towers)
training_text = []
validation_text = []
test_text = []
for i in range(num_towers):
    training_text.append(data[i*training_size:(i+1)*training_size])
    validation_text.append(data[num_towers*training_size + i*validation_size: \
                                num_towers*training_size + (i+1)*validation_size])
    test_text.append(data[num_towers*(training_size + validation_size) + i*test_size: \
                          num_towers*(training_size + validation_size) + (i+1)*test_size])

In [ ]:
print('Vocabulary Size: %d' % vocabulary_size)

# Initiate graph
if rnn_flg == 1:
    # Use SRN
    graph = srn_graph(cluster_spec, num_gpus, hidden_size, vocabulary_size, num_training_unfoldings, 
                      optimization_frequency, clip_norm, momentum, batch_size, num_validation_unfoldings)
elif rnn_flg == 2:
    # Use LSTM
    graph = lstm_graph(cluster_spec, num_gpus, hidden_size, vocabulary_size, num_training_unfoldings, 
                       optimization_frequency, clip_norm, momentum, batch_size, num_validation_unfoldings)
elif rnn_flg == 3:
    # Use SCRN
    graph = scrn_graph(cluster_spec, num_gpus, alpha, hidden_size, state_size, vocabulary_size, 
                       num_training_unfoldings, optimization_frequency, clip_norm, momentum, batch_size, 
                       num_validation_unfoldings)
    
# Optimize graph
graph.optimization(learning_rate, learning_decay, num_epochs, summary_frequency, training_text, validation_text)

Vocabulary Size: 44612
Training Batch Generator:
     Tower: 0
          Input Text Size: 13913351
          Cut Text Size: 13912000
          Subtext Size: 434750
          Dropped Text Size: 1351
          Effective Batch Size: 1600
          Number of Batches: 8695
Validation Batch Generator:
     Tower: 0
          Input Text Size: 1545927
          Cut Text Size: 1545000
          Subtext Size: 1545000
          Dropped Text Size: 927
          Effective Batch Size: 1000
          Number of Batches: 1545
Initialized
Epoch: 1  Learning Rate: 0.05
     Total Batches: 500  Current Batch: 500  Cost: 7.62
